In [6]:
# Importa find_best_hyperparameter e pandas
from find_best_hyperparameters import AtividadeTres
import pandas as pd

# Carrega Atividade 3
a3 = AtividadeTres(random_state=21)

Classe iniciada com semente 21!


In [7]:
# Leitura da base CSTR
BASE='CSTR'
cstr = a3.load_data(BASE + '.csv')
cstr.head()

,file_name,text,class
0,126.txt,Rhetorical (Rhet) is a programming / knowledge...,ArtificiallIntelligence
1,5.txt,Reduction is the operation of transforming a p...,ArtificiallIntelligence
2,48.txt,"For years, researchers have used knowledge-int...",ArtificiallIntelligence
3,81.txt,Proceedings of a workshop held in conjunction ...,ArtificiallIntelligence
4,25.txt,The Medication Advisor is the latest project o...,ArtificiallIntelligence


In [8]:
# Separação dos dados de treino e teste
train, test, series = a3.split_data(data=cstr, text_col='text', class_col='class', test_size=0.2)

# Salva resultados
a3.save_data_frame(train, BASE + '_train.csv')
a3.save_data_frame(test, BASE + '_test.csv')

# Mostra primeiras linhas do teste
test.head()

Dados divididos com tamanho do teste 0.20 e semente 21!
Chaves do dicionário: dict_keys(['X_train', 'X_test', 'y_train', 'y_test'])
Arquivo out/CSTR_train.csv salvo com sucesso!
Arquivo out/CSTR_test.csv salvo com sucesso!


,text,class
61,This study of the Fall 2002 Computer Programmi...,ArtificiallIntelligence
297,We study the complexity of influencing electio...,Theory
55,We describe two domain-independent temporal re...,ArtificiallIntelligence
243,We argue for transactions as the synchronizati...,Systems
95,"In this paper, we present the results for sema...",ArtificiallIntelligence


In [4]:
serie.keys()

dict_keys(['X_train', 'X_test', 'y_train', 'y_test'])